In [6]:
from ipynb.fs.full.DBMySQLConnection import getDB, connectDB
import pandas as pd
import plotly.express as px
import dask_ml.cluster
import copy
import math 
import spotipy.util as util
import spotipy
import os
%run '../Experiments/Utils.ipynb'

In [7]:
token = util.prompt_for_user_token('31z7enbknuggbtpidrsxziofxmlm','user-library-read',client_id='87b4599f0c0a4eb8bb6d8f6e3528df94',client_secret='3f93fff3d1ce434ea325760554fb772e',redirect_uri='http://localhost:8080')

sp = spotipy.Spotify(auth=token)

In [8]:

coldplaysongs = pd.read_csv('../Experiments/data/coldplaySongs.csv')

In [9]:
coldplaysongs.iloc[[1,3,7,8,9,11,12,13,17,18,19,20,23],:].id_track

1     Coldplay - The Scientist (Cello & Piano) - Bro...
3     Coldplay - The Scientist (Official Video) (152...
7     GABRIELLA - Coldplay - The Scientist (Cover) (...
8     GLEE - The Scientist (Full Performance) (Offic...
9     GLEE - The Scientist (Full Performance) HD (15...
11    Savanna Leigh - The Scientist (Lyrics) (152kbi...
12       The Scientist (152kbit_Opus)-converted.mp3.csv
13    The Scientist - Coldplay (Boyce Avenue feat. H...
17    The Scientist - Coldplay Cover (ft. Kina Grann...
18    The Scientist - Coldplay _ Alex Goot & Jada Fa...
19    The Scientist - The Petersens (LIVE) (128kbit_...
20    Wicker Park - The Scientist (Coldplay) (192kbi...
23        gleechapversionthescientist-converted.mp3.csv
Name: id_track, dtype: object

In [10]:
coldplaysongs


,Unnamed: 0,id_chord,id_track,chords,orig_key
0,0,5000001,Coldplay - The Scientist (Acoustic Cover) (128...,"Am,F,C,G,Am,F,C,G,Am,F,C,G,Am,F,C,Am,F,C,Am,F,...",E
1,1,5000002,Coldplay - The Scientist (Cello & Piano) - Bro...,"Am,F,C,Am,F,C,F,C,F,C,F,C,F,C,F,C,F,C,G,Bdim,C...",F
2,2,5000003,Coldplay - The Scientist (Live in Madrid 2011)...,"Am,F,C,G,Am,F,C,G,Am,F,C,G,Am,F,C,Am,F,C,Am,F,...",F
3,3,5000004,Coldplay - The Scientist (Official Video) (152...,"Am,F,C,Am,F,C,Am,F,C,Am,F,C,F,C,F,C,G,G,C,F,C,...",F
4,4,5000005,Coldplay - The Scientist (Piano Cover) (152kbi...,"E,Bm,C,G,Em,C,G,C,G,C,G,C,G,Em,G,C,G,F#m,C,G,B...",A#
5,5,5000006,Coldplay - The Scientist in the Live Lounge (1...,"Bm,Am,F,F,C,Am,F,C,F,C,F,C,F,C,F,C,G,C,F,C,G,A...",F
6,6,5000007,Coldplay The Scientist Frankfurt 5-7-22 (128kb...,"Am,F,C,Am,F,C,C,F,C,F,C,F,C,F,C,F,C,F,C,C,G,G,...",F
7,7,5000008,GABRIELLA - Coldplay - The Scientist (Cover) (...,"C,Am,C,Am,F,C,Am,Dm,F,C,Em,Am,F,C,Am,F,C,Em,F,...",E
8,8,5000009,GLEE - The Scientist (Full Performance) (Offic...,"Gm,D#,A#,A#,D#,A#,D#,A#,D#,A#,F,D#,A#,F,Adim,A...",A
9,9,5000010,GLEE - The Scientist (Full Performance) HD (15...,"Gm,D#,A#,A#,D#,A#,A#,D#,A#,D#,A#,D#,A#,F,F,A#,...",A


In [11]:
ids =[
    '025vpSSyndhVZezl8rUF6L', 
    '6TyaYGQKCnRLVTOQlXvYAA', 
    '68iMdxXn1timrxURm9dR8C', 
    '6f46TyGZm5oeKIP54PLvql', 
    '34dnNAUoIPcwnK0RtVMBWZ', 
    '49ilSrZNQcle6PYw8rtvzM', 
    '66Z3LdLjtwfIdHFF0wZa5B', 
    '2LbHYfTYW6TipHInIjqdaQ',
    '49Nzxf1LskunwmZ6EyWUew',
    '5FoM2ZWwhgTxHFFh2stUw6',
    '7tn1tcxenMwDXrYTOjS3z1',
]
features = sp.audio_features(ids)


In [12]:
features

[{'danceability': 0.255,
  'energy': 0.343,
  'key': 5,
  'loudness': -8.117,
  'mode': 1,
  'speechiness': 0.031,
  'acousticness': 0.927,
  'instrumentalness': 0.867,
  'liveness': 0.164,
  'valence': 0.116,
  'tempo': 140.527,
  'type': 'audio_features',
  'id': '025vpSSyndhVZezl8rUF6L',
  'uri': 'spotify:track:025vpSSyndhVZezl8rUF6L',
  'track_href': 'https://api.spotify.com/v1/tracks/025vpSSyndhVZezl8rUF6L',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/025vpSSyndhVZezl8rUF6L',
  'duration_ms': 295471,
  'time_signature': 4},
 {'danceability': 0.273,
  'energy': 0.205,
  'key': 4,
  'loudness': -11.893,
  'mode': 1,
  'speechiness': 0.0316,
  'acousticness': 0.941,
  'instrumentalness': 0.000682,
  'liveness': 0.171,
  'valence': 0.0796,
  'tempo': 148.225,
  'type': 'audio_features',
  'id': '6TyaYGQKCnRLVTOQlXvYAA',
  'uri': 'spotify:track:6TyaYGQKCnRLVTOQlXvYAA',
  'track_href': 'https://api.spotify.com/v1/tracks/6TyaYGQKCnRLVTOQlXvYAA',
  'analysis_url': 'https:

In [13]:
featureDf = pd.DataFrame( features, columns = ['id_spotify_track', 'id_track', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature', 'name',
       'popularity', 'album_name', 'release_date', 'year', 'mood'])
featureDf['id_spotify_track'] = ids


In [14]:
featureDf.head()

,id_spotify_track,id_track,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,...,valence,tempo,duration_ms,time_signature,name,popularity,album_name,release_date,year,mood
0,025vpSSyndhVZezl8rUF6L,NaN,0.255,0.343,5,-8.117,1,0.0310,0.927,0.867000,...,0.1160,140.527,295471,4,NaN,NaN,NaN,NaN,NaN,NaN
1,6TyaYGQKCnRLVTOQlXvYAA,NaN,0.273,0.205,4,-11.893,1,0.0316,0.941,0.000682,...,0.0796,148.225,309074,4,NaN,NaN,NaN,NaN,NaN,NaN
2,68iMdxXn1timrxURm9dR8C,NaN,0.506,0.481,2,-6.098,1,0.0264,0.482,0.000000,...,0.2240,148.626,254413,4,NaN,NaN,NaN,NaN,NaN,NaN
3,6f46TyGZm5oeKIP54PLvql,NaN,0.495,0.489,2,-6.083,1,0.0270,0.497,0.000000,...,0.2480,148.928,255293,4,NaN,NaN,NaN,NaN,NaN,NaN
4,34dnNAUoIPcwnK0RtVMBWZ,NaN,0.453,0.295,7,-12.881,1,0.0291,0.518,0.000110,...,0.1400,146.022,276575,4,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
featureDf.at[0,'mode'] = 1
featureDf.head()

,id_spotify_track,id_track,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,...,valence,tempo,duration_ms,time_signature,name,popularity,album_name,release_date,year,mood
0,025vpSSyndhVZezl8rUF6L,NaN,0.255,0.343,5,-8.117,1,0.0310,0.927,0.867000,...,0.1160,140.527,295471,4,NaN,NaN,NaN,NaN,NaN,NaN
1,6TyaYGQKCnRLVTOQlXvYAA,NaN,0.273,0.205,4,-11.893,1,0.0316,0.941,0.000682,...,0.0796,148.225,309074,4,NaN,NaN,NaN,NaN,NaN,NaN
2,68iMdxXn1timrxURm9dR8C,NaN,0.506,0.481,2,-6.098,1,0.0264,0.482,0.000000,...,0.2240,148.626,254413,4,NaN,NaN,NaN,NaN,NaN,NaN
3,6f46TyGZm5oeKIP54PLvql,NaN,0.495,0.489,2,-6.083,1,0.0270,0.497,0.000000,...,0.2480,148.928,255293,4,NaN,NaN,NaN,NaN,NaN,NaN
4,34dnNAUoIPcwnK0RtVMBWZ,NaN,0.453,0.295,7,-12.881,1,0.0291,0.518,0.000110,...,0.1400,146.022,276575,4,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
ids =[
    '025vpSSyndhVZezl8rUF6L', 
    '6TyaYGQKCnRLVTOQlXvYAA', 
    '68iMdxXn1timrxURm9dR8C', 
    '6f46TyGZm5oeKIP54PLvql', 
    '34dnNAUoIPcwnK0RtVMBWZ', 
    '49ilSrZNQcle6PYw8rtvzM', 
    '66Z3LdLjtwfIdHFF0wZa5B', 
    '2LbHYfTYW6TipHInIjqdaQ',
    '49Nzxf1LskunwmZ6EyWUew',
    '5FoM2ZWwhgTxHFFh2stUw6',
    '7tn1tcxenMwDXrYTOjS3z1',
]

for id,value in enumerate(ids):
    song = sp.audio_analysis(value)
    track = song['track']
    featureDf.at[id,'tempo'] = track['tempo']
    featureDf.at[id,'time_signature'] = track['time_signature']
    featureDf.at[id,'key'] = track['key']
    featureDf.at[id,'mode'] = track['mode']


In [17]:
featureDf['mode'].head()

0    1
1    1
2    1
3    1
4    1
Name: mode, dtype: int64

In [18]:
idSongs = coldplaysongs.iloc[[1,7,8, 23, 17, 19, 20, 13, 18, 11, 12],:]
featureDf['id_track'] = idSongs.id_track.tolist()
featureDf['id_chord'] = [5000001,5000007,5000008, 5000023, 5000017, 5000019, 5000020, 5000013, 5000018, 5000011, 5000012]
featureDf['id_chord']

0     5000001
1     5000007
2     5000008
3     5000023
4     5000017
5     5000019
6     5000020
7     5000013
8     5000018
9     5000011
10    5000012
Name: id_chord, dtype: int64

In [19]:
featureDf

,id_spotify_track,id_track,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,...,tempo,duration_ms,time_signature,name,popularity,album_name,release_date,year,mood,id_chord
0,025vpSSyndhVZezl8rUF6L,Coldplay - The Scientist (Cello & Piano) - Bro...,0.255,0.343,5,-8.117,1,0.0310,0.927,0.867000,...,140.527,295471,4,NaN,NaN,NaN,NaN,NaN,NaN,5000001
1,6TyaYGQKCnRLVTOQlXvYAA,GABRIELLA - Coldplay - The Scientist (Cover) (...,0.273,0.205,4,-11.893,1,0.0316,0.941,0.000682,...,148.225,309074,4,NaN,NaN,NaN,NaN,NaN,NaN,5000007
2,68iMdxXn1timrxURm9dR8C,GLEE - The Scientist (Full Performance) (Offic...,0.506,0.481,2,-6.098,1,0.0264,0.482,0.000000,...,148.626,254413,4,NaN,NaN,NaN,NaN,NaN,NaN,5000008
3,6f46TyGZm5oeKIP54PLvql,gleechapversionthescientist-converted.mp3.csv,0.495,0.489,2,-6.083,1,0.0270,0.497,0.000000,...,148.928,255293,4,NaN,NaN,NaN,NaN,NaN,NaN,5000023
4,34dnNAUoIPcwnK0RtVMBWZ,The Scientist - Coldplay Cover (ft. Kina Grann...,0.453,0.295,7,-12.881,1,0.0291,0.518,0.000110,...,146.022,276575,4,NaN,NaN,NaN,NaN,NaN,NaN,5000017
5,49ilSrZNQcle6PYw8rtvzM,The Scientist - The Petersens (LIVE) (128kbit_...,0.549,0.283,9,-11.006,1,0.0284,0.661,0.000008,...,76.992,303426,4,NaN,NaN,NaN,NaN,NaN,NaN,5000019
6,66Z3LdLjtwfIdHFF0wZa5B,Wicker Park - The Scientist (Coldplay) (192kbi...,0.493,0.270,5,-11.910,1,0.0278,0.244,0.000000,...,73.001,305868,4,NaN,NaN,NaN,NaN,NaN,NaN,5000020
7,2LbHYfTYW6TipHInIjqdaQ,The Scientist - Coldplay (Boyce Avenue feat. H...,0.334,0.252,8,-11.776,1,0.0287,0.722,0.000000,...,153.879,251514,4,NaN,NaN,NaN,NaN,NaN,NaN,5000013
8,49Nzxf1LskunwmZ6EyWUew,The Scientist - Coldplay _ Alex Goot & Jada Fa...,0.525,0.292,9,-8.809,1,0.0260,0.910,0.000000,...,75.009,248227,4,NaN,NaN,NaN,NaN,NaN,NaN,5000018
9,5FoM2ZWwhgTxHFFh2stUw6,Savanna Leigh - The Scientist (Lyrics) (152kbi...,0.503,0.123,10,-14.189,1,0.0333,0.879,0.000000,...,145.843,233076,4,NaN,NaN,NaN,NaN,NaN,NaN,5000011


In [20]:

featureDf['mood'] = featureDf.apply(lambda x: setMoodLabel(x), axis=1)
featureDf['mood'] = featureDf.apply(lambda x: setMoodNumber(x), axis=1)
featureDf['mood']

0     2
1     1
2     2
3     2
4     2
5     2
6     2
7     2
8     2
9     1
10    1
Name: mood, dtype: int64

In [21]:
featureDf.to_csv('../Experiments/data/coldplaySongsSpotifyFeatures.csv')

In [58]:
featureDf

,id_spotify_track,id_track,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,...,tempo,duration_ms,time_signature,name,popularity,album_name,release_date,year,mood,id_chord
0,025vpSSyndhVZezl8rUF6L,Coldplay - The Scientist (Cello & Piano) - Bro...,0.255,0.343,5,-8.117,1,0.0310,0.927,0.867000,...,140.527,295471,4,NaN,NaN,NaN,NaN,NaN,NaN,5000001
1,6TyaYGQKCnRLVTOQlXvYAA,GABRIELLA - Coldplay - The Scientist (Cover) (...,0.273,0.205,4,-11.893,1,0.0316,0.941,0.000682,...,148.225,309074,4,NaN,NaN,NaN,NaN,NaN,NaN,5000007
2,68iMdxXn1timrxURm9dR8C,GLEE - The Scientist (Full Performance) (Offic...,0.506,0.481,2,-6.098,1,0.0264,0.482,0.000000,...,148.626,254413,4,NaN,NaN,NaN,NaN,NaN,NaN,5000008
3,6f46TyGZm5oeKIP54PLvql,gleechapversionthescientist-converted.mp3.csv,0.495,0.489,2,-6.083,1,0.0270,0.497,0.000000,...,148.928,255293,4,NaN,NaN,NaN,NaN,NaN,NaN,5000023
4,34dnNAUoIPcwnK0RtVMBWZ,The Scientist - Coldplay Cover (ft. Kina Grann...,0.453,0.295,7,-12.881,1,0.0291,0.518,0.000110,...,146.022,276575,4,NaN,NaN,NaN,NaN,NaN,NaN,5000017
5,49ilSrZNQcle6PYw8rtvzM,The Scientist - The Petersens (LIVE) (128kbit_...,0.549,0.283,9,-11.006,1,0.0284,0.661,0.000008,...,76.992,303426,4,NaN,NaN,NaN,NaN,NaN,NaN,5000019
6,66Z3LdLjtwfIdHFF0wZa5B,Wicker Park - The Scientist (Coldplay) (192kbi...,0.493,0.270,5,-11.910,1,0.0278,0.244,0.000000,...,73.001,305868,4,NaN,NaN,NaN,NaN,NaN,NaN,5000020
7,2LbHYfTYW6TipHInIjqdaQ,The Scientist - Coldplay (Boyce Avenue feat. H...,0.334,0.252,8,-11.776,1,0.0287,0.722,0.000000,...,153.879,251514,4,NaN,NaN,NaN,NaN,NaN,NaN,5000013
8,49Nzxf1LskunwmZ6EyWUew,The Scientist - Coldplay _ Alex Goot & Jada Fa...,0.525,0.292,9,-8.809,1,0.0260,0.910,0.000000,...,75.009,248227,4,NaN,NaN,NaN,NaN,NaN,NaN,5000018
9,5FoM2ZWwhgTxHFFh2stUw6,Savanna Leigh - The Scientist (Lyrics) (152kbi...,0.503,0.123,10,-14.189,1,0.0333,0.879,0.000000,...,145.843,233076,4,NaN,NaN,NaN,NaN,NaN,NaN,5000011


In [ ]:
# Regardless of the DB Engine fill df variable

cn = connectDB()
df = pd.read_sql("""
select distinct id_spotify
from spotify_tracks
""", con=cn)
cn.close()

In [ ]:
listTracks = df['id_spotify'].tolist()

In [ ]:
listTracks[0:0+50]

In [ ]:
def insertAudioFeatures(listTracks):
    try:
        cn = connectDB()
        if cn.is_connected():
            cr = cn.cursor()

            ## Add here the for each
            
            for i in range(0, len(listTracks), 50):
                print('::: LIST ::::', listTracks[i:i+50])
                new_features = sp.audio_features(listTracks[i:i+50])              
                for track in new_features:
                    if(track):
                        print( track['id'])
                        sql = "INSERT INTO spotify_audio_features (id_track, danceability, energy, `key`, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, duration_ms, time_signature) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                        val = (track['id'], track['danceability'], track['energy'], track['key'], track['loudness'], track['mode'], track['speechiness'], track['acousticness'], track['instrumentalness'], track['liveness'], track['valence'], track['tempo'], track['duration_ms'], track['time_signature'])
                        cr.execute(sql, val)     
                    else:
                        print('not features found for the track')
               

    except Error as e:
        print("Error while connecting to MySQL", e)
    finally:
        if cn.is_connected():
            cn.commit()
            cr.close()
            cn.close()
            print("MySQL cn is closed")


In [ ]:
insertAudioFeatures(listTracks)